# Simulation with AVISO fields - Bigger Domain - NRT



### Weekly backward simulations within the Oleander domain (Gulf Stream). Running days= 7. Near-real-time simulations.


In [1]:
%matplotlib inline
from parcels import Field, FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter
from datetime import timedelta as delta

from matplotlib              import dates as mdates
from datetime                import datetime 
import pickle

## Save the outputs in this directory

In [2]:
dirsave    = '/Users/bbarcelo/HOME_SCIENCE/Data/2019_SSS_Ladvection/sim_weekly_all_Bigger_Domain/sim_outputs/' #save outputs directory

## Altimetry data directory

In [3]:
dir_netcdf    = '/Users/bbarcelo/HOME_SCIENCE/Data/2019_SSS/SLA_GS_big_region/' 
# Data from 2019-05-14 to 2020-01-15
file_currents = 'dataset-duacs-nrt-global-merged-allsat-phy-l4_1583920119159.nc'
print(file_currents)

   
# current field file starts at this date, check for each new file the first date in altimetry_check_first_day.py
date_ori     = mdates.date2num(datetime(2019, 5, 14, 0, 0, 0))  
date_alt_fin = mdates.date2num(datetime(2020, 1, 15, 0, 0, 0)) #end day of delayed-time fiel  

dataset-duacs-nrt-global-merged-allsat-phy-l4_1583920119159.nc


## Running times to test

In [4]:
rund = [7,1] # run times for each simulation

## Define the particles release grid

In [5]:
# DOMAIN FOR weekly simulations BIGGER DOMAIN
lonmin, lonmax = -82, -63 #-74, -66 #-74, -68
latmin, latmax =  25, 46 #35, 40 #37, 40 

dx = 0.04

lons, lats = np.meshgrid(np.arange(360+lonmin, 360+lonmax, dx), np.arange(latmin, latmax, dx))

# we need to have lons, lats and times in a 1d vector (flatten)
lonf, latf = lons.flatten(), lats.flatten()

## Allow particles to leave the domain

Allow particles to leave the domain. We need to tell Parcels that particles that leave the domain need to be deleted. We do that using a `Recovery Kernel`, which will be invoked when a particle encounters an `ErrorOutOfBounds` error:

In [6]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()

## File with the simulation dates

In [16]:
dir_dic    = '/Users/bbarcelo/HOME_SCIENCE/Data/2019_SSS_Ladvection/sim_weekly_all/'
file_dic = 'sim_weekly_relase_dates_corrected_gaps.pkl'

f = open(dir_dic + file_dic, 'rb')
dict_release_dates = pickle.load(f)
f.close() 

dates_release_all = dict_release_dates['date_release'] #- 0.5 #starting at 00:00 instead than at 12:00

# only dates in which we have delayed-time altimetry data 

ind_dt = np.where(dates_release_all>date_ori+6) # we need 7 days of currents before the release day

dates_release_pre = dates_release_all.astype(int)[ind_dt]
#dates_release_pre = list(dates_release_all.astype(int)[ind_dt])

print(dates_release_pre)

# now is not necessary to re-run all, only after and including 2019-6-11

date_to_start = mdates.date2num(datetime(2019, 6, 11, 0, 0, 0)) 

ind_rerun = np.where(dates_release_pre >= date_to_start) 

dates_release = list(dates_release_pre[ind_rerun])

print(dates_release)


[737200 737207 737214 737221 737273 737280 737287 737294 737301 737308
 737315 737322 737329 737336 737343 737350 737357 737364 737371 737378
 737385 737397 737404 737411 737418 737425 737432 737439 737446 737453
 737460 737467 737474 737481 737488 737495 737502]
[737221, 737273, 737280, 737287, 737294, 737301, 737308, 737315, 737322, 737329, 737336, 737343, 737350, 737357, 737364, 737371, 737378, 737385, 737397, 737404, 737411, 737418, 737425, 737432, 737439, 737446, 737453, 737460, 737467, 737474, 737481, 737488, 737495, 737502]


In [17]:
dates_release_all.shape


(252,)

In [18]:
len(dates_release)


34

In [19]:
date_to_start

737221.0

## Loop, do simulations for each date in the list

In [20]:
for date in dates_release:

    #date_time_obj = datetime.strptime(date, '%Y%m%d')
    date_time_obj = mdates.num2date(date)


    year  = date_time_obj.year
    month = date_time_obj.month
    day   = date_time_obj.day
    
    
    print('simulation date...', year, month, day)
    
    # Define end date of the simulation (backward simulation)
    date_fin = mdates.date2num(datetime(year, month, day, 0, 0, 0))  # Final time of the backward simulation
    
    
    # First, define the names of the files containing the zonal (U) and meridional (V) velocities. 
    filenames = {'U': dir_netcdf + file_currents,
                 'V': dir_netcdf + file_currents}
    
    # Then, define a dictionary of the variables (`U` and `V`) and dimensions 
    # (`lon`, `lat` and `time`; note that in this case there is no `depth` because the data 
    # is only for the surface of the ocean).

    # Left: name of the variable that Parcels needs
    # Right: name of the variable in the nc file 
    
    variables = {'U': 'ugos',
                 'V': 'vgos'}
    dimensions = {'lat': 'latitude', # it has to be like this so Parcels can detect this is latitude
                  'lon': 'longitude', # lon is from 0 to 360!
                  'time': 'time'} # it has to be like this to not have an error
    
    
    # Finally, read in the fieldset using the `FieldSet.from_netcdf` function with the 
    # above-defined `filenames`, `variables` and `dimensions`
    
    fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)
    
    # Allow particles to leave the domain. We need to tell Parcels that particles that 
    # leave the domain need to be deleted. We do that using a `Recovery Kernel`, which 
    # will be invoked when a particle encounters an `ErrorOutOfBounds` error:
    
    #def DeleteParticle(particle, fieldset, time):
      #particle.delete()
        
    # Do all simulations in a loop to test different running times
    # Running particles in backward time is extremely simple: just provide a dt < 0.
    
    daysf = date_fin - date_ori # number of days between date_ori and date_fin    

    for rd in rund: # number of days of the simulation (run time)   
        daysi = daysf # the release day is the final date
        timef = np.ones(lonf.shape) * delta(days=daysi).total_seconds()  # release particle daysi later from 1-Jan-2017
    
        pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lonf, lat=latf, time=timef) 

        savefilename = 'sim_weekly_back_alt_BD_'+mdates.num2date(date_fin).strftime("%Y%m%d")+'_' +'%02i'% rd +'days_nrt.nc' 
        
        print('Starting simulation for run day...', rd)

        pset.execute(AdvectionRK4,                    # the kernel (which defines how particles move)
             runtime=timedelta(days=int(rd)),   # the total length of the run
             dt= -timedelta(hours=3),           # negative timestep for backward run
             output_file=pset.ParticleFile(name= dirsave + savefilename, outputdt=timedelta(hours=12)), # the file name and the time step of the outputs
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})  # the recovery kernel

        print('Simulation saved as... ', savefilename)
        
    print('')
    print('Simulations finished for this date!!')
    print('')
    
print('SIMULATIONS FINISHED!!')    

simulation date... 2019 6 11
Starting simulation for run day... 7


INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/1r/g9d70q7x59s6sd00xl3mmyfw0000gn/T/parcels-501/75399675b7129f98b58bb0b63937854b.so
100% (604800.0 of 604800.0) |############| Elapsed Time: 1:11:50 Time:  1:11:50


Simulation saved as...  sim_weekly_back_alt_BD_20190611_07days_nrt.nc
Starting simulation for run day... 1


INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/1r/g9d70q7x59s6sd00xl3mmyfw0000gn/T/parcels-501/d47bfd0f9632f422ac2b6a98cc44ea68.so
100% (86400.0 of 86400.0) |##############| Elapsed Time: 0:05:26 Time:  0:05:26


Simulation saved as...  sim_weekly_back_alt_BD_20190611_01days_nrt.nc

Simulations finished for this date!!

simulation date... 2019 8 2
Starting simulation for run day... 7


...

Simulation saved as...  sim_weekly_back_alt_BD_20200115_07days_nrt.nc
Starting simulation for run day... 1


INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/1r/g9d70q7x59s6sd00xl3mmyfw0000gn/T/parcels-501/6536bb5df4ad4b4ac8037d100a95746a.so
100% (86400.0 of 86400.0) |##############| Elapsed Time: 0:05:22 Time:  0:05:22


Simulation saved as...  sim_weekly_back_alt_BD_20200115_01days_nrt.nc

Simulations finished for this date!!

simulation date... 2020 1 22
Starting simulation for run day... 7


INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/1r/g9d70q7x59s6sd00xl3mmyfw0000gn/T/parcels-501/450f1818e4f59041f7e67556b912178a.so


TimeExtrapolationError: U sampled outside time domain at time 2020-01-22T00:00:00.000000000. Try setting allow_time_extrapolation to True

In [130]:
pset.show(field=fieldset.U)

In [134]:
plotTrajectoriesFile(savefilename);

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/bbarcelo/HOME_SCIENCE/Scripts/2019_SSS_Ladvection/simulation_notebooks/sim_Oleander_back_alt_20170212_07days.nc'